In [1]:
import pandas as pd
import os
import re
import torch
from tqdm import tqdm
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM

In [2]:
BASE_DIR = os.getcwd()
DATA_DIR = os.path.join(BASE_DIR, "data")
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

In [3]:
data_path = os.path.join(DATA_DIR, "train.csv")
origin_df = pd.read_csv(data_path)

In [ ]:
new_df = origin_df.copy()
for idx, text in enumerate(new_df['text']) : 
    new_df.loc[new_df['text'] == text, 'text'] = re.sub(r'[^가-힣ㅏ-ㅣ ]+', ' ', text).strip()

new_df

,ID,text,target
0,ynat-v1_train_00000,정 파 미사 이용기간 단 분종 보,4
1,ynat-v1_train_00001,찰 국 로 한 회장 송,3
2,ynat-v1_train_00002,김정 자주통일 새 열 나가야 보,2
3,ynat-v1_train_00003,갤노트 주말 만대 개통 시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,대선 앞두고 단 발 비해 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨 점만점에 점도,6
2796,ynat-v1_train_02796,삼성 갤럭시 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,텔레그램 한 등 서 시간 다운 버 정 종 보,4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,1


In [40]:
new_df

,ID,text,target
0,ynat-v1_train_00000,"한국 축구 대표팀, 월드컵 예선전서 2-0 승리",4
1,ynat-v1_train_00001,"정부, 올림",3
2,ynat-v1_train_00002,"한국, 월드컵 8강 진출 확정",2
3,ynat-v1_train_00003,갤노트 주말 100만 대 개통 시장은 불법 보조금 얼룩,5
4,ynat-v1_train_00004,대선 앞두고 단 발 비해 감시 강화,6
...,...,...,...
2795,ynat-v1_train_02795,트럼프 폭스뉴스 앵커들 충성도 점수매겨 점만점에 점도,6
2796,ynat-v1_train_02796,삼성 갤럭시 정식 출시 첫 주말 이통시장 잠잠,2
2797,ynat-v1_train_02797,텔레그램 한 등 서 시간 다운 버 정 종 보,4
2798,ynat-v1_train_02798,인터뷰 류현진 친구에게 안타 맞는 것 싫어해 승부는 냉정,1


In [4]:
# 모델과 토크나이저 로드
model_name = 'yanolja/EEVE-Korean-Instruct-10.8B-v1.0'
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name, torch_dtype=torch.float16)
model.to(DEVICE)

tokenizer_config.json:   0%|          | 0.00/2.04k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/2.18M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/557 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/704 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/35.8k [00:00<?, ?B/s]

...

model-00003-of-00005.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00004-of-00005.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00005-of-00005.safetensors:   0%|          | 0.00/1.88G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/5 [00:00<?, ?it/s]

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(40960, 4096)
    (layers): ModuleList(
      (0-47): 48 x LlamaDecoderLayer(
        (self_attn): LlamaAttention(
          (q_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (k_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (v_proj): Linear(in_features=4096, out_features=1024, bias=False)
          (o_proj): Linear(in_features=4096, out_features=4096, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (up_proj): Linear(in_features=4096, out_features=14336, bias=False)
          (down_proj): Linear(in_features=14336, out_features=4096, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
        (post_attention_layernorm): LlamaRMSNorm((4096,), eps=1e-05)
      )
    )
    (norm):

In [5]:
for tidx, noised_text in enumerate(new_df['text']) :
    prompt = f"""당신은 한국어 신문 기사 헤드라인 복구 전문가입니다. 주어진 불완전한 헤드라인을 분석하고, 누락된 문자를 추론하여 원래의 완전한 헤드라인을 복구해야 합니다. 
                주어진 헤드라인에 대해 가장 적절하고 자연스러운 복구 결과를 제시하세요. 복구 시 다음 지침을 따르세요:

                1. 한국어 신문 기사 헤드라인의 일반적인 스타일과 어조를 유지하세요.
                2. 누락된 숫자, 고유명사, 또는 중요 단어를 추론하여 채워넣으세요.
                3. 문맥을 고려하여 가장 그럴듯한 내용으로 복구하세요.
                4. 복구된 헤드라인은 완전하고 의미가 통해야 합니다.
                5. 원본 헤드라인의 길이와 구조를 최대한 유지하세요.
                
                이제 다음의 불완전한 헤드라인을 복구해주세요:
                불완전한 헤드라인: {noised_text}
                복구된 헤드라인:"""
                
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_text = generated_text.split("생성된 헤드라인:")[-1].split('\n')[0]
    new_df.loc[tidx, 'text'] = cleaned_text
    print("origin : ", noised_text)
    print("cleaned : ", cleaned_text)

new_df.to_csv(f'./data/fill_mask/train_filled.csv', index=False)


NameError: name 'new_df' is not defined

In [ ]:
new_df = origin_df.copy()
for idx, text in enumerate(new_df['text']) : 
    new_df.loc[new_df['text'] == text, 'text'] = re.sub(r'[^가-힣ㅏ-ㅣ ]+', ' ', text).strip()

for tidx, noised_text in enumerate(new_df['text']) :
    prompt = f"""당신은 한국어 신문 기사 헤드라인 생성 전문가입니다. 주어진 불완전한 문자들을 포함하는 새로운 신문 기사 헤드라인을 생성하세요. 
                주어진 불완전한 문장의 문자들을 가장 많이 포함하는 자연스러운 신문기사 헤드라인을 생성하세요. 생성 시 다음 지침을 따르세요:

                1. 한국어 신문 기사 헤드라인의 일반적인 스타일과 어조를 유지하세요.
                2. 기존의 문장의 단어와 문자를 최대한 많이 포함해야합니다.
                3. 문맥을 고려하여 가장 그럴듯한 내용으로 복구하세요.
                4. 생성된 헤드라인은 완전하고 의미가 통해야 합니다.
                5. 주어진 문장이 많이 훼손되어 복구가 어려운 경우 완전히 새로운 헤드라인을 생성하세요.
                
                이제 다음의 문장을 이용해 새로운 헤드라인을 생성해주세요:
                불완전한 헤드라인: {noised_text}
                생성된 헤드라인:"""

    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    with torch.no_grad():
        outputs = model.generate(
            **inputs,
            max_new_tokens=50,
            num_return_sequences=1,
            temperature=0.7,
            top_p=0.95,
            do_sample=True
        )

    generated_text = tokenizer.decode(outputs[0], skip_special_tokens=True)
    cleaned_text = generated_text.split("생성된 헤드라인:")[-1].split('\n')[0]
    new_df.loc[tidx, 'text'] = cleaned_text
    print("origin : ", noised_text)
    print("cleaned : ", cleaned_text)

new_df.to_csv(f'./data/fill_mask/train_generated.csv', index=False)


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  정   파  미사    이용기간   단   분종 보
cleaned :   "정부의 미사용 기간제 보육원, 분종별 이용률 높다"


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  찰 국  로 한  회장    송
cleaned :   찰 스와거트 국무장관, 한중 회담에서 협력 논의


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


...

origin :  강 감독  로축구연맹 회부
cleaned :   "강 감독, 축구연맹에 회부되어 징계위원회에 회부되다"


Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


origin :  축구 인천    운드  영
cleaned :   축구 인천 유나이티드, 우승을 향해 힘차게 달립니다!


KeyboardInterrupt: 

: 